d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# 2.6 Broadcast Joins

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this notebook you:<br>
* Compare and contrast broadcast and shuffle joins
* Examine the Physical Plan that is generated for your queries
* Optimize joins using Broadcast joins

In [0]:
%run ../Includes/Classroom-Setup

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Broadcast and Shuffle Joins

-sandbox
## Standard Join

* In a standard join, **ALL** the data is shuffled.
* This can be really expensive.

<img src="https://files.training.databricks.com/images/eLearning/ucdavis/join-shuffle.png" alt="Join" style="max-height:500px"/>

-sandbox
## Broadcast Join
* In a Broadcast Join, only the "small" data is moved.
* It duplicates the "small" data across all executors.
* But the "big" data is left untouched.
* If the "small" data is small enough, this can be **VERY** efficient.

<img src="https://files.training.databricks.com/images/eLearning/ucdavis/join-broadcast.png" alt="Broadcast Join" style="max-height:500px"/>

In brief, joins are _very_ expensive operations.  This becomes increasingly apparent in big data environments where joins involve transferring data across a network.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Examining Physical Plans

Let's make sure our data is accessible.

In [0]:
%sql
USE databricks;

DESCRIBE fireCalls

col_name,data_type,comment
Call Number,int,null
Unit ID,string,null
Incident Number,int,null
Call Type,string,null
Call Date,string,null
Watch Date,string,null
Received DtTm,string,null
Entry DtTm,string,null
Dispatch DtTm,string,null
Response DtTm,string,null


Now create the table `fireCallsParquet`.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fireCallsParquet
USING Parquet 
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-8p.parquet"
  )

We can join these two datasets and examine the physical plan (how data is physically affected) using `EXPLAIN`.

In [0]:
%sql
EXPLAIN 
  SELECT * 
  FROM fireCalls 
  JOIN fireCallsParquet on fireCalls.`Call Number` = fireCallsParquet.`Call_Number`

plan
"== Physical Plan == *(3) SortMergeJoin [Call Number#369], [Call_Number#298], Inner :- Sort [Call Number#369 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(Call Number#369, 200) : +- *(1) Project [Call Number#369, Unit ID#370, Incident Number#371, Call Type#372, Call Date#373, Watch Date#374, Received DtTm#375, Entry DtTm#376, Dispatch DtTm#377, Response DtTm#378, On Scene DtTm#379, Transport DtTm#380, Hospital DtTm#381, Call Final Disposition#382, Available DtTm#383, Address#384, City#385, Zipcode of Incident#386, Battalion#387, Station Area#388, Box#389, Original Priority#390, Priority#391, Final Priority#392, ... 10 more fields] : +- *(1) Filter isnotnull(Call Number#369) : +- *(1) FileScan csv databricks.firecalls[Call Number#369,Unit ID#370,Incident Number#371,Call Type#372,Call Date#373,Watch Date#374,Received DtTm#375,Entry DtTm#376,Dispatch DtTm#377,Response DtTm#378,On Scene DtTm#379,Transport DtTm#380,Hospital DtTm#381,Call Final Disposition#382,Available DtTm#383,Address#384,City#385,Zipcode of Incident#386,Battalion#387,Station Area#388,Box#389,Original Priority#390,Priority#391,Final Priority#392,... 10 more fields] Batched: false, DataFilters: [isnotnull(Call Number#369)], Format: CSV, Location: InMemoryFileIndex[dbfs:/mnt/davis/fire-calls/fire-calls-truncated-comma.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Call Number)], ReadSchema: struct<Call Number:int,Unit ID:string,Incident Number:int,Call Type:string,Call Date:string,Watch... +- Sort [Call_Number#298 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(Call_Number#298, 200) +- *(2) Project [Call_Number#298, Unit_ID#299, Incident_Number#300, Call_Type#301, Call_Date#302, Watch_Date#303, Received_DtTm#304, Entry_DtTm#305, Dispatch_DtTm#306, Response_DtTm#307, On_Scene_DtTm#308, Transport_DtTm#309, Hospital_DtTm#310, Call_Final_Disposition#311, Available_DtTm#312, Address#313, City#314, Zipcode_of_Incident#315, Battalion#316, Station_Area#317, Box#318, Original_Priority#319, Priority#320, Final_Priority#321, ... 10 more fields] +- *(2) Filter isnotnull(Call_Number#298) +- *(2) FileScan parquet [Call_Number#298,Unit_ID#299,Incident_Number#300,Call_Type#301,Call_Date#302,Watch_Date#303,Received_DtTm#304,Entry_DtTm#305,Dispatch_DtTm#306,Response_DtTm#307,On_Scene_DtTm#308,Transport_DtTm#309,Hospital_DtTm#310,Call_Final_Disposition#311,Available_DtTm#312,Address#313,City#314,Zipcode_of_Incident#315,Battalion#316,Station_Area#317,Box#318,Original_Priority#319,Priority#320,Final_Priority#321,... 10 more fields] Batched: true, DataFilters: [isnotnull(Call_Number#298)], Format: Parquet, Location: InMemoryFileIndex[dbfs:/mnt/davis/fire-calls/fire-calls-8p.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(Call_Number)], ReadSchema: struct<Call_Number:int,Unit_ID:string,Incident_Number:int,Call_Type:string,Call_Date:string,Watch..."


#### Automatic and Manual broadcasting

- Depending on size of the data that is being loaded into Spark, Spark uses internal heuristics to decide how to join that data to other data.
- Automatic broadcast depends on `spark.sql.autoBroadcastJoinThreshold`
    - The setting configures the **maximum size in bytes** for a table that will be broadcast to all worker nodes when performing a join 
    - Default is 10MB

- A `broadcast` function can be used in Spark to instruct Catalyst that it should probably broadcast one of the tables that is being joined. 

If the `broadcast` hint isn't used, but one side of the join is small enough (i.e., its size is below the threshold), that data source will be read into
the Driver and broadcast to all Executors.

In [0]:
%python
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

Out[1]: '10485760'

Now take a look at the physical plan when we broadcast one of the datasets.  The broadcast join hint is going to operate like a SQL hint, but Spark will still parse this even though it is commented out.

In [0]:
%sql
EXPLAIN 
  SELECT /*+ BROADCAST(fireCalls) */ * 
  FROM fireCalls 
  JOIN fireCallsParquet on fireCalls.`Call Number` = fireCallsParquet.`Call_Number`


plan
"== Physical Plan == *(2) BroadcastHashJoin [Call Number#369], [Call_Number#298], Inner, BuildLeft :- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, true] as bigint))) : +- *(1) Project [Call Number#369, Unit ID#370, Incident Number#371, Call Type#372, Call Date#373, Watch Date#374, Received DtTm#375, Entry DtTm#376, Dispatch DtTm#377, Response DtTm#378, On Scene DtTm#379, Transport DtTm#380, Hospital DtTm#381, Call Final Disposition#382, Available DtTm#383, Address#384, City#385, Zipcode of Incident#386, Battalion#387, Station Area#388, Box#389, Original Priority#390, Priority#391, Final Priority#392, ... 10 more fields] : +- *(1) Filter isnotnull(Call Number#369) : +- *(1) FileScan csv databricks.firecalls[Call Number#369,Unit ID#370,Incident Number#371,Call Type#372,Call Date#373,Watch Date#374,Received DtTm#375,Entry DtTm#376,Dispatch DtTm#377,Response DtTm#378,On Scene DtTm#379,Transport DtTm#380,Hospital DtTm#381,Call Final Disposition#382,Available DtTm#383,Address#384,City#385,Zipcode of Incident#386,Battalion#387,Station Area#388,Box#389,Original Priority#390,Priority#391,Final Priority#392,... 10 more fields] Batched: false, DataFilters: [isnotnull(Call Number#369)], Format: CSV, Location: InMemoryFileIndex[dbfs:/mnt/davis/fire-calls/fire-calls-truncated-comma.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Call Number)], ReadSchema: struct<Call Number:int,Unit ID:string,Incident Number:int,Call Type:string,Call Date:string,Watch... +- *(2) Project [Call_Number#298, Unit_ID#299, Incident_Number#300, Call_Type#301, Call_Date#302, Watch_Date#303, Received_DtTm#304, Entry_DtTm#305, Dispatch_DtTm#306, Response_DtTm#307, On_Scene_DtTm#308, Transport_DtTm#309, Hospital_DtTm#310, Call_Final_Disposition#311, Available_DtTm#312, Address#313, City#314, Zipcode_of_Incident#315, Battalion#316, Station_Area#317, Box#318, Original_Priority#319, Priority#320, Final_Priority#321, ... 10 more fields] +- *(2) Filter isnotnull(Call_Number#298) +- *(2) FileScan parquet [Call_Number#298,Unit_ID#299,Incident_Number#300,Call_Type#301,Call_Date#302,Watch_Date#303,Received_DtTm#304,Entry_DtTm#305,Dispatch_DtTm#306,Response_DtTm#307,On_Scene_DtTm#308,Transport_DtTm#309,Hospital_DtTm#310,Call_Final_Disposition#311,Available_DtTm#312,Address#313,City#314,Zipcode_of_Incident#315,Battalion#316,Station_Area#317,Box#318,Original_Priority#319,Priority#320,Final_Priority#321,... 10 more fields] Batched: true, DataFilters: [isnotnull(Call_Number#298)], Format: Parquet, Location: InMemoryFileIndex[dbfs:/mnt/davis/fire-calls/fire-calls-8p.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(Call_Number)], ReadSchema: struct<Call_Number:int,Unit_ID:string,Incident_Number:int,Call_Type:string,Call_Date:string,Watch..."


-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>